In [50]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
import imutils
import pprint
from matplotlib import pyplot as plt
import pandas as pd

In [7]:
image = cv2.imread("face.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.3,
    minNeighbors=3,
    minSize=(30, 30)
)

for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    roi_color = image[y:y + h, x:x + w]
    b = str(w) + str(h) + '_faces.jpg'
    cv2.imwrite(b, roi_color)


In [48]:
def extractSkin(image):
    
    img = image.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_threshold = np.array([0, 48, 80], dtype=np.uint8)
    upper_threshold = np.array([20, 255, 255], dtype=np.uint8)
    skinMask = cv2.inRange(img, lower_threshold, upper_threshold)
    skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)
    skin = cv2.bitwise_and(img, img, mask=skinMask)
    return cv2.cvtColor(skin, cv2.COLOR_HSV2BGR)

def removeBlack(estimator_labels, estimator_cluster):

    hasBlack = False
    occurance_counter = Counter(estimator_labels)
    def compare(x, y): return Counter(x) == Counter(y)
    for x in occurance_counter.most_common(len(estimator_cluster)):
        color = [int(i) for i in estimator_cluster[x[0]].tolist()]
        if compare(color, [0, 0, 0]) == True:
            del occurance_counter[x[0]]
            hasBlack = True
            estimator_cluster = np.delete(estimator_cluster, x[0], 0)
            break
    return (occurance_counter, estimator_cluster, hasBlack)

def getColorInformation(estimator_labels, estimator_cluster, hasThresholding=False):

    occurance_counter = None
    colorInformation = []
    hasBlack = False
    if hasThresholding == True:
        (occurance, cluster, black) = removeBlack(
            estimator_labels, estimator_cluster)
        occurance_counter = occurance
        estimator_cluster = cluster
        hasBlack = black
    else:
        occurance_counter = Counter(estimator_labels)
    totalOccurance = sum(occurance_counter.values())
    for x in occurance_counter.most_common(len(estimator_cluster)):
        index = (int(x[0]))
        index = (index-1) if ((hasThresholding & hasBlack)& (int(index) != 0)) else index
        color = estimator_cluster[index].tolist()
        color_percentage = (x[1]/totalOccurance)
        colorInfo = {"cluster_index": index, "color": color,
                     "color_percentage": color_percentage}
        colorInformation.append(colorInfo)
    return colorInformation

def extractDominantColor(image, number_of_colors=5, hasThresholding=False):

    if hasThresholding == True:
        number_of_colors += 1
    img = image.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.reshape((img.shape[0]*img.shape[1]), 3)
    estimator = KMeans(n_clusters=number_of_colors, random_state=0)
    estimator.fit(img)
    colorInformation = getColorInformation(estimator.labels_, estimator.cluster_centers_, hasThresholding)
    return colorInformation

def getcol(x):
    comp = {}
    append = {}
    for i in x:
        comp[i['cluster_index']] = i['color_percentage']
        append[i['cluster_index']] = i['color']
    m = max(comp)
    col =[]
    col.append(append[m])
    return col

def pipeline(image):
    image=cv2.imread(image)
    skin = extractSkin(image)
    x = extractDominantColor(skin, hasThresholding=True)
    y = getcol(x)
    return y

In [51]:
pro = pd.read_csv("product.csv")
sh = pd.read_csv("shades.csv")

In [61]:
d = []
n = []
for i in sh.index:
    try:
        l = sh['Image'][i]
        m = "images/"+l
        print(m)
        d.append(pipeline(m))
    except:
        n.append(i)

images/New York Fit Me Matte+Poreless Liquid Foundation - 128 Warm Nude 30ml-115 Ivory.jpg
images/New York Fit Me Matte+Poreless Liquid Foundation - 128 Warm Nude 30ml-Classic Ivory.jpg
images/New York Fit Me Matte+Poreless Liquid Foundation - 128 Warm Nude 30ml-Natural Tan.jpg
images/New York Fit Me Matte+Poreless Liquid Foundation - 128 Warm Nude 30ml-Warm Nude 128.jpg


<ipython-input-48-a5bbea6ed560>:58: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  estimator.fit(img)
<ipython-input-48-a5bbea6ed560>:58: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  estimator.fit(img)
<ipython-input-48-a5bbea6ed560>:58: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  estimator.fit(img)
<ipython-input-48-a5bbea6ed560>:58: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  estimator.fit(img)


images/New York Fit Me Matte+Poreless Liquid Foundation - 128 Warm Nude 30ml-Natural Beige.jpg
images/Perfecting Liquid Foundation - Natural Marble 27ml-Natural Coral.jpg
images/Perfecting Liquid Foundation - Natural Marble 27ml-Natural Shell.jpg
images/Perfecting Liquid Foundation - Natural Marble 27ml-Natural Marble.jpg
images/Perfecting Liquid Foundation - Natural Marble 27ml-Natural Pearl.jpg
images/New York Fit Me! Matte+Poreless Liquid Foundation - Natural Beige 220 30ml-115 Ivory.jpg
images/New York Fit Me! Matte+Poreless Liquid Foundation - Natural Beige 220 30ml-Classic Ivory.jpg


<ipython-input-48-a5bbea6ed560>:58: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  estimator.fit(img)
<ipython-input-48-a5bbea6ed560>:58: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  estimator.fit(img)
<ipython-input-48-a5bbea6ed560>:58: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  estimator.fit(img)


images/New York Fit Me! Matte+Poreless Liquid Foundation - Natural Beige 220 30ml-Natural Tan.jpg
images/New York Fit Me! Matte+Poreless Liquid Foundation - Natural Beige 220 30ml-Warm Nude 128.jpg
images/New York Fit Me! Matte+Poreless Liquid Foundation - Natural Beige 220 30ml-Natural Beige.jpg
images/Paris True Match Liquid Foundation - N1 Ivory 30ml-Rich Caramel.jpg


...

images/Ultra Definition Liquid Foundation - Chocolate Mousse-Pecan.jpg
images/Ultra Definition Liquid Foundation - Chocolate Mousse-Brownie.jpg
images/Ultra Definition Liquid Foundation - Chocolate Mousse-Beige.jpg
images/Ultra Definition Liquid Foundation - Chocolate Mousse-Beige Caramel.jpg
images/Ultra Definition Liquid Foundation - Chocolate Mousse-Milk Cake.jpg
images/Ultra Definition Liquid Foundation - Chocolate Mousse-Pralines Cake.jpg
images/Ultra Definition Liquid Foundation - Chocolate Mousse-Cheese Cake.jpg
images/Ultra Definition Liquid Foundation - Chocolate Mousse-Peanut Butter.jpg
images/Ultra Definition Liquid Foundation - Chocolate Mousse-Chocolate Mousse.jpg
images/Ultra Definition Liquid Foundation - Chocolate Mousse-Honey.jpg
images/Ultra Definition Liquid Foundation - Chocolate Mousse-Fudge.jpg
images/Weightless Matte Finish Mini Foundation Ivory 01 18ml-Ivory.jpg
images/Weightless Matte Finish Mini Foundation Ivory 01 18ml-Beige.jpg


<ipython-input-48-a5bbea6ed560>:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  estimator.fit(img)
<ipython-input-48-a5bbea6ed560>:58: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  estimator.fit(img)
<ipython-input-48-a5bbea6ed560>:58: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  estimator.fit(img)


images/Beige Spotlight Glow Stick 14 g-Beige.jpg
images/Spotlight Glow Stick 14 g-Beige.jpg
images/Beige Spotlight Glow Foundation Stick 14 g-Beige.jpg
images/Ultra Definition Liquid Foundation- Honey 30 ml-Pecan.jpg


<ipython-input-48-a5bbea6ed560>:58: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  estimator.fit(img)


images/Ultra Definition Liquid Foundation- Honey 30 ml-Brownie.jpg
images/Ultra Definition Liquid Foundation- Honey 30 ml-Beige.jpg
images/Ultra Definition Liquid Foundation- Honey 30 ml-Beige Caramel.jpg
images/Ultra Definition Liquid Foundation- Honey 30 ml-Milk Cake.jpg
images/Ultra Definition Liquid Foundation- Honey 30 ml-Pralines Cake.jpg
images/Ultra Definition Liquid Foundation- Honey 30 ml-Cheese Cake.jpg
images/Ultra Definition Liquid Foundation- Honey 30 ml-Peanut Butter.jpg
images/Ultra Definition Liquid Foundation- Honey 30 ml-Chocolate Mousse.jpg
images/Ultra Definition Liquid Foundation- Honey 30 ml-Honey.jpg
images/Ultra Definition Liquid Foundation- Honey 30 ml-Fudge.jpg
images/Ultra Definition Liquid Foundation - Cheesecake 30 ml-Pecan.jpg
images/Ultra Definition Liquid Foundation - Cheesecake 30 ml-Brownie.jpg
images/Ultra Definition Liquid Foundation - Cheesecake 30 ml-Beige.jpg
images/Ultra Definition Liquid Foundation - Cheesecake 30 ml-Beige Caramel.jpg
images/Ul

In [69]:
for i in n:
    sh = sh.drop(i)

sh['Color'] = d
sh.to_csv('colors.csv', sep=',', mode='a')